In [1]:
#Packages import
import numpy as np
import pandas as pd

In [11]:
def volts(IVCurve): #input: csv file containing IV curve. Returns Vp, Vf
    IV = pd.read_csv(IVCurve,skiprows=61).to_numpy()
    print(IV)
    

In [13]:
volts("LP/Trace_10p8mTorr_400W_1.csv")

[['1' '0' '-20.0111427307129' '-0.000778974092099816']
 ['2' '0.021707' '-19.9612579345703' '-0.000778915651608258']
 ['3' '0.043413' '-19.9110641479492' '-0.000778702553361654']
 ...
 ['Mean' '10.9933813236763' '4.99920588079672' '0.00387733259188641']
 ['StdDev' '6.41733942346349' '14.4598383245109' '0.00686730481708026']
 ['CV' '58.3745731592384%' '289.242705127528%' '177.114153979222%']]
